# 1. Modelová úloha: dosažitelnost vrcholů v grafu

Hledáme, do kterých vrcholů se dostaneme z vrcholu `0` při nejvýše `n` krocích.

- Graf má vrcholy `0` až `n-1`.
- Hrany jsou dvojice `(u, v)`.
- Jeden krok znamená přechod po jedné hraně.

## 1.1 Vstupní data

Nejdřív vygenerujeme jednoduchý náhodný neorientovaný graf.
- Pro každou dvojici vrcholů rozhodneme, zda mezi nimi hrana vznikne.
- Pravděpodobnost nastavíme tak, aby průměrný stupeň vrcholu byl přibližně `d`.

In [ ]:
import random
n = 10

def vygeneruj_graf(n, d = 3):
    V = [i for i in range(n)]
    prob = d / n
    E = []
    for v_i in range(n):
        for v_j in range(v_i + 1, n):
            if random.random() < prob:
                E.append((v_i, v_j))
    return V, E

V, E = vygeneruj_graf(n)

In [ ]:
print(V)
print(E)

## 1.2 Vizualizace přes `networkx`

In [ ]:
# !pip install networkx

In [ ]:
# plot graph with vertices V and edges E
# showing vertices with numbers and connections as lines
import matplotlib.pyplot as plt
import networkx as nx

G = nx.Graph()
G.add_nodes_from(V)
G.add_edges_from(E)
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos)
nx.draw_networkx_edges(G, pos)
nx.draw_networkx_labels(G, pos)
plt.show()


## 1.3 Vizualizace přes GraphViz

Alternativní vizualizace přes `graphviz` knihovnu.

In [ ]:
import graphviz

# Create a new graph
graph = graphviz.Graph()

# Add vertices to the graph
_ = [graph.node(str(vertex)) for vertex in V]

# Add edges to the graph
_ = [graph.edge(str(edge[0]), str(edge[1])) for edge in E]

# Render and display the graph
graph


## 1.4 První návrh algoritmu

- Udržujeme množinu dosažitelných vrcholů.
- V každém kroku projdeme všechny hrany a přidáme nové dosažitelné sousedy.

In [ ]:
def reachable_in_n_steps(edges, n):
    reachable = set()
    reachable.add(0)
    for i in range(n):
        new_reachable = set()
        for v in reachable:
            for e in edges:
                if e[0] == v:
                    new_reachable.add(e[1])
                if e[1] == v:
                    new_reachable.add(e[0])
        reachable = reachable.union(new_reachable)
    return sorted(reachable)

In [ ]:
reachable_in_n_steps(E, 2)

## 1.5 Větší graf

In [ ]:
V, E = vygeneruj_graf(2000)

Jak dlouho to potrvá pro větší vstup?

In [ ]:
%time _ = reachable_in_n_steps(E, 20)

In [ ]:
res1 = reachable_in_n_steps(E, 20)

## 1.6 Profilování

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f reachable_in_n_steps reachable_in_n_steps(E, 20)

Hrany procházíme mnohokrát. To je hlavní důvod, proč první verze škáluje špatně.

## 1.7 Algoritmická optimalizace

Před úpravou implementace si ujasníme logiku:
- Nechceme opakovaně zpracovávat stejné informace.
- Má smysl sledovat jen vrcholy přidané v posledním kroku.
- Jakmile už nové vrcholy nepřibývají, můžeme skončit.

Změny v nové verzi:
- držíme seznam dosud nezpracovaných hran,
- v každém kroku rozšiřujeme jen z nově dosažených vrcholů,
- jakmile nepřibude žádný vrchol, končíme.

Vstupní seznam hran se uvnitř funkce kopíruje, protože ho průběžně měníme.

In [ ]:
def reachable_in_n_steps_v2(edges_in: list, n: int):
    edges = edges_in.copy()  # kopie hran, abychom je mohli mazat
    reachable = set()
    reachable.add(0)  # začínáme ve vrcholu 0
    newly_reachable = reachable.copy()  # vrcholy, které jsme právě přidali

    for _ in range(n):  # počet kroků
        next_reachable = set()  # vrcholy dosažitelné z newly_reachable

        for e_idx, e in list(enumerate(edges))[::-1]:  # pozpátku kvůli mazání
            for v in newly_reachable:
                if e[0] == v:
                    next_reachable.add(e[1])
                    edges.pop(e_idx)
                    break
                if e[1] == v:
                    next_reachable.add(e[0])
                    edges.pop(e_idx)
                    break

        newly_reachable = next_reachable.difference(reachable)
        reachable = reachable.union(next_reachable)
        if not newly_reachable:
            break

    return sorted(reachable)

In [ ]:
%time res2 = reachable_in_n_steps_v2(E, 20)

Tohle už je znatelné zrychlení.

Ověříme, že výsledek zůstává stejný:

In [ ]:
import numpy as np
res2 = reachable_in_n_steps_v2(E, 20)
np.array_equal(np.array(res1), np.array(res2))

In [ ]:
%lprun -f reachable_in_n_steps_v2 reachable_in_n_steps_v2(E, 5)

## 1.8 Další algoritmická optimalizace: reprezentace grafu

Místo seznamu hran použijeme pro každý vrchol seznam sousedů. Tím odstraníme opakované procházení všech hran.

In [ ]:
def reachable_in_n_steps_v3(edges_in: list, n: int):
    n_vertices = max([max(e) for e in edges_in]) + 1
    edges = [[] for _ in range(n_vertices)]

    for e in edges_in:
        edges[e[0]].append(e[1])
        edges[e[1]].append(e[0])

    reachable = set()
    reachable.add(0)
    newly_reachable = reachable.copy()

    for _ in range(n):
        next_reachable = set()
        for v in newly_reachable:
            [next_reachable.add(soused) for soused in edges[v]]

        newly_reachable = next_reachable.difference(reachable)
        reachable = reachable.union(next_reachable)
        if not newly_reachable:
            break

    return sorted(reachable)

In [ ]:
%timeit res2 = reachable_in_n_steps_v3(E, 20)

To je další výrazné zrychlení.

Zkontrolujeme, že výstup zůstává stejný.

In [ ]:
res2 = reachable_in_n_steps_v3(E, 20)
np.array_equal(np.array(res1), np.array(res2))

In [ ]:
%lprun -f reachable_in_n_steps_v3 reachable_in_n_steps_v3(E, 20)

## 1.9 Implementační optimalizace: NumPy a bool masky

Zkusíme stejný princip zapsat přes NumPy pole a bool masky místo Python setů.

Vstup si připravíme i jako NumPy pole:

In [ ]:
E_np = np.array(E)
E_np.shape

In [ ]:
def reachable_in_n_steps_np(edges_in, n):
    n_vertices = np.max(edges_in) + 1  # počet vrcholů

    # počet odchozích hran (kumulativně) pro každý vrchol (první záznam fixně 0)
    edges_index_sousedu = np.zeros(n_vertices + 1, dtype=np.int32)  
    # neprve nasčítáme počty hran
    for i in range(edges_in.shape[0]):
        edges_index_sousedu[edges_in[i, 0] + 1] += 1
        edges_index_sousedu[edges_in[i, 1] + 1] += 1
    # pak provedeme kumulativní součet
    for i in range(1, n_vertices + 1):
        edges_index_sousedu[i] += edges_index_sousedu[i - 1]

    # indexy sousedních vrcholů pro každý vrchol (tvar dle CSR/CSC formátu)
    edges_sousede = np.zeros(edges_in.size, dtype=np.int32)
    edges_tmp_index = edges_index_sousedu.copy() # kopie pro držení indexů k zápisu
    for i in range(edges_in.shape[0]):  # pro všechny hrany zapiš vrcholy souseda
        edges_sousede[edges_tmp_index[ edges_in[i, 0]]] = edges_in[i, 1]
        edges_tmp_index[ edges_in[i, 0]] += 1
        edges_sousede[edges_tmp_index[edges_in[i, 1]]] =  edges_in[i, 0]
        edges_tmp_index[edges_in[i, 1]] += 1

    reachable = np.zeros(n_vertices, dtype=np.bool_)  # maska dosažitelných vrcholů
    reachable[0] = True  # začínáme ve vrcholu 0
    newly_reachable = reachable.copy()  # vrcholy, které jsme právě přidali

    for _ in range(n):
        next_reachable = np.zeros(n_vertices, dtype=np.bool_)
        for v in np.where(newly_reachable)[0]:  # cyklus jen přes indexy nově dosažitelných vrcholů
            # přidáme všechny sousedy dle edges_sousede
            next_reachable[edges_sousede[edges_index_sousedu[v]:edges_index_sousedu[v + 1]]] = True

        # pouze nově přidané vrcholy = přidané teď a zároveň dosud nedosažitelné
        newly_reachable = np.logical_and(next_reachable, np.logical_not(reachable))

        # všechny dosažitelné vrcholy = dosud dosažitelné nebo nově dosažitelné
        reachable = np.logical_or(reachable, next_reachable)

        if not np.any(newly_reachable):
            break

    return np.where(reachable)[0]

In [ ]:
%timeit res3 = reachable_in_n_steps_np(E_np, 20)

In [ ]:
res3 = reachable_in_n_steps_np(E_np, 20)
np.array_equal(np.array(res1), np.array(res3))

Tato varianta bez kompilace nepřinesla zásadní zlepšení.

Podíváme se, kde tráví čas.

In [ ]:
%lprun -f reachable_in_n_steps_np reachable_in_n_steps_np(E_np, 20)

Zkusíme stejnou NumPy variantu zkompilovat pomocí Numby.

## 1.10 Implementační optimalizace II: Numba

### 1.10.1 NumPy verze

In [ ]:
from numba import jit
import numpy as np

reachable_in_n_steps_np_numba = jit(reachable_in_n_steps_np, nopython=True)


In [ ]:
%time res4 = reachable_in_n_steps_np_numba(E_np, 20)

In [ ]:
%timeit res4 = reachable_in_n_steps_np_numba(E_np, 20)

Tady je zrychlení velmi výrazné.

Ověříme shodu výsledků:

In [ ]:
res4 = reachable_in_n_steps_np_numba(E_np, 20)
np.array_equal(np.array(res1), np.array(res4))

### 1.10.2 Python verze (listy a sety)

Pro srovnání zkusíme zkompilovat i variantu `v2`.

In [ ]:
reachable_in_n_steps_v2_numba = jit(reachable_in_n_steps_v2, nopython=True)


In [ ]:
%time _ = reachable_in_n_steps_v2_numba(E, 20)

In [ ]:
%timeit _ = reachable_in_n_steps_v2_numba(E, 20)

Variantu `v3` nelze přímočaře přeložit přes Numbu, protože pracuje s vnořenými dynamickými Python strukturami.

In [ ]:
# reachable_in_n_steps_v3_numba = jit(reachable_in_n_steps_v3, nopython=True)
# _ = reachable_in_n_steps_v3_numba(E, 20)

## 1.11 Alternativa: matice sousednosti (SciPy)

Další možnost je reprezentovat graf řídkou maticí sousednosti.

- `CSR` (Compressed Sparse Row) ukládá matici po řádcích a je vhodná pro násobení `A @ v`.
- `CSC` (Compressed Sparse Column) ukládá matici po sloupcích a je vhodná hlavně pro operace po sloupcích.

V této úloze používáme `CSR`, protože v každém kroku násobíme matici vektorem dosažitelných vrcholů.


In [ ]:
from scipy.sparse import csr_matrix


def build_adjacency_csr(edges):
    n_vertices = int(np.max(edges)) + 1
    n_edges = edges.shape[0]

    # Neorientovaný graf: každou hranu (u, v) zapíšeme jako (u, v) i (v, u).
    rows = np.empty(2 * n_edges, dtype=np.int32)
    cols = np.empty(2 * n_edges, dtype=np.int32)
    rows[:n_edges] = edges[:, 0]
    rows[n_edges:] = edges[:, 1]
    cols[:n_edges] = edges[:, 1]
    cols[n_edges:] = edges[:, 0]

    data = np.ones(2 * n_edges, dtype=np.uint8)
    adjacency_csr = csr_matrix((data, (rows, cols)), shape=(n_vertices, n_vertices), dtype=np.uint8)
    adjacency_csr.sum_duplicates()
    adjacency_csr.data[:] = 1
    adjacency_csr.sort_indices()
    return adjacency_csr


def reachable_in_n_steps_scipy(adjacency_csr, n):
    n_vertices = adjacency_csr.shape[0]

    reachable = np.zeros(n_vertices, dtype=np.bool_)
    frontier = np.zeros(n_vertices, dtype=np.bool_)
    reachable[0] = True
    frontier[0] = True

    for _ in range(n):
        frontier = adjacency_csr.dot(frontier).astype(np.bool_, copy=False)
        frontier = np.logical_and(frontier, np.logical_not(reachable))
        if not np.any(frontier):
            break
        reachable = np.logical_or(reachable, frontier)

    return np.where(reachable)[0]


In [ ]:
%timeit _ = build_adjacency_csr(E_np)

adjacency_csr = build_adjacency_csr(E_np)
%timeit res5 = reachable_in_n_steps_scipy(adjacency_csr, 20)


Nejdřív je vidět cena sestavení matice, pak cena samotného šíření.

Zkontrolujeme shodu výsledků:

In [ ]:
adjacency_csr = build_adjacency_csr(E_np)
res5 = reachable_in_n_steps_scipy(adjacency_csr, 20)
np.array_equal(np.array(res1), res5)


In [ ]:
# profilování sestavení matice
%lprun -f build_adjacency_csr build_adjacency_csr(E_np)

# profilování samotného průchodu
%lprun -f reachable_in_n_steps_scipy reachable_in_n_steps_scipy(adjacency_csr, 20)


## 1.12 Benchmark nejlepších variant

Pro větší testy budeme chtít i rychlejší generování grafů.

In [ ]:
import numba
vygeneruj_graf = numba.jit(vygeneruj_graf)


### 1.12.1 Závislost času na počtu kroků `n`

Vynecháme nejpomalejší nezkompilované varianty.

In [ ]:
import matplotlib.pyplot as plt
import time

num_vert = 1500
V, E = vygeneruj_graf(num_vert)
E_np = np.array(E)
adjacency_csr = build_adjacency_csr(E_np)

n_list = [2**i for i in range(0, 9)]
times_v2_numba = []  # reachable_in_n_steps_v2_numba
times_v3 = []  # reachable_in_n_steps_v3
times_np = []  # reachable_in_n_steps_np
times_np_numba = []  # reachable_in_n_steps_np_numba
times_scipy = []  # reachable_in_n_steps_scipy

for n in n_list:
    start = time.time()
    _ = reachable_in_n_steps_v2_numba(E, n)
    end = time.time()
    times_v2_numba.append(end - start)

    start = time.time()
    _ = reachable_in_n_steps_v3(E, n)
    end = time.time()
    times_v3.append(end - start)

    start = time.time()
    _ = reachable_in_n_steps_np(E_np, n)
    end = time.time()
    times_np.append(end - start)

    start = time.time()
    _ = reachable_in_n_steps_np_numba(E_np, n)
    end = time.time()
    times_np_numba.append(end - start)

    start = time.time()
    _ = reachable_in_n_steps_scipy(adjacency_csr, n)
    end = time.time()
    times_scipy.append(end - start)

    print(n, times_v2_numba[-1], times_np[-1], times_np_numba[-1], times_scipy[-1])

plt.figure(figsize=(10, 10))
plt.loglog(n_list, times_v2_numba, label='v2 Numba')
plt.loglog(n_list, times_v3, label='v3')
plt.loglog(n_list, times_np, label='np')
plt.loglog(n_list, times_np_numba, label='np numba')
plt.loglog(n_list, times_scipy, label='scipy (CSR prebuilt)')

plt.xlabel('Number of steps')
plt.ylabel('Time [s]')
plt.title(f"Závislost výpočetního času na počtu kroků pro velikost grafu {num_vert}.")
plt.grid()
plt.legend()


Porovnáme jen nejrychlejší varianty pro širší rozsah kroků.

In [ ]:
import matplotlib.pyplot as plt
import time

num_vert = 1500
V, E = vygeneruj_graf(num_vert, 1)
E_np = np.array(E)
adjacency_csr = build_adjacency_csr(E_np)

n_list = [2**i for i in range(0, 13)]
times_v3 = []  # reachable_in_n_steps_v3
times_np_numba = []  # reachable_in_n_steps_np_numba
times_scipy = []  # reachable_in_n_steps_scipy

for n in n_list:
    start = time.time()
    _ = reachable_in_n_steps_v3(E, n)
    end = time.time()
    times_v3.append(end - start)

    start = time.time()
    _ = reachable_in_n_steps_np_numba(E_np, n)
    end = time.time()
    times_np_numba.append(end - start)

    start = time.time()
    _ = reachable_in_n_steps_scipy(adjacency_csr, n)
    end = time.time()
    times_scipy.append(end - start)

    print(n, times_v3[-1], times_np_numba[-1], times_scipy[-1])

plt.figure(figsize=(10, 10))
plt.loglog(n_list, times_v3, label='v3')
plt.loglog(n_list, times_np_numba, label='np numba')
plt.loglog(n_list, times_scipy, label='scipy (CSR prebuilt)')

plt.xlabel('Number of steps')
plt.ylabel('Time [s]')
plt.title(f"Závislost výpočetního času na počtu kroků pro velikost grafu {num_vert}.")
plt.grid()
plt.legend()


### 1.12.2 Závislost času na počtu vrcholů a hran

In [ ]:
import time
import matplotlib.pyplot as plt

n = 10  # počet kroků
num_vert_list = [2**i for i in range(1, 11)]
times_v2_numba = []  # reachable_in_n_steps_v2_numba
times_v3 = []  # reachable_in_n_steps_v3
times_np = []  # reachable_in_n_steps_np
times_np_numba = []  # reachable_in_n_steps_numba
times_scipy = []  # reachable_in_n_steps_scipy

for num_vert in num_vert_list:
    V, E = vygeneruj_graf(num_vert)
    E_np = np.array(E)
    adjacency_csr = build_adjacency_csr(E_np)

    start = time.time()
    _ = reachable_in_n_steps_v2_numba(E, n)
    end = time.time()
    times_v2_numba.append(end - start)

    start = time.time()
    _ = reachable_in_n_steps_v3(E, n)
    end = time.time()
    times_v3.append(end - start)

    start = time.time()
    _ = reachable_in_n_steps_np(E_np, n)
    end = time.time()
    times_np.append(end - start)

    start = time.time()
    _ = reachable_in_n_steps_np_numba(E_np, n)
    end = time.time()
    times_np_numba.append(end - start)

    start = time.time()
    _ = reachable_in_n_steps_scipy(adjacency_csr, n)
    end = time.time()
    times_scipy.append(end - start)

    print(num_vert, times_v2_numba[-1], times_np[-1], times_np_numba[-1], times_scipy[-1])

plt.figure(figsize=(10, 10))
plt.loglog(num_vert_list, times_v2_numba, label='v2 numba')
plt.loglog(num_vert_list, times_v3, label='v3')
plt.loglog(num_vert_list, times_np, label='np')
plt.loglog(num_vert_list, times_np_numba, label='np numba')
plt.loglog(num_vert_list, times_scipy, label='scipy (CSR prebuilt)')

plt.xlabel('Graph size')
plt.ylabel('Time [s]')
plt.title(f"Závislost výpočetního času na velikosti grafu pro počet kroků {n}.")
plt.grid()
plt.legend()


I zde zůstáváme jen u rychlejších variant.

In [ ]:
import time
import matplotlib.pyplot as plt

n = 10  # počet kroků
num_vert_list = [2**i for i in range(1, 18)]
times_v3 = []  # reachable_in_n_steps_v3
times_np_numba = []  # reachable_in_n_steps_numba
times_scipy = []  # reachable_in_n_steps_scipy

for num_vert in num_vert_list:
    V, E = vygeneruj_graf(num_vert)
    E_np = np.array(E)
    adjacency_csr = build_adjacency_csr(E_np)

    start = time.time()
    _ = reachable_in_n_steps_v3(E, n)
    end = time.time()
    times_v3.append(end - start)

    start = time.time()
    _ = reachable_in_n_steps_np_numba(E_np, n)
    end = time.time()
    times_np_numba.append(end - start)

    start = time.time()
    _ = reachable_in_n_steps_scipy(adjacency_csr, n)
    end = time.time()
    times_scipy.append(end - start)

    print(num_vert, times_v3[-1], times_np_numba[-1], times_scipy[-1])

plt.figure(figsize=(10, 10))
plt.loglog(num_vert_list, times_v3, label='v3')
plt.loglog(num_vert_list, times_np_numba, label='np numba')
plt.loglog(num_vert_list, times_scipy, label='scipy (CSR prebuilt)')

plt.xlabel('Graph size')
plt.ylabel('Time [s]')
plt.title(f"Závislost výpočetního času na velikosti grafu pro počet kroků {n}.")
plt.grid()
plt.legend()
